<a href="https://colab.research.google.com/github/vincent0965/Colab_note/blob/master/news_kind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
glob.glob("drive/My Drive/*")

In [8]:
import zipfile
path = "drive/My Drive/chinese_news_test.zip"
f = zipfile.ZipFile(path)
f.extractall()
path = "drive/My Drive/chinese_news_trans.zip"
f = zipfile.ZipFile(path)
f.extractall()

In [7]:
import os
import glob
import pandas as pd

def read_data(base):
    contents, targets = [], []
    basedn = os.path.join(base, "*")
    for dn in glob.glob(basedn):
        upper = os.path.join(dn, "*.TXT")
        lower = os.path.join(dn, "*.txt")
        for fn in glob.glob(upper) + glob.glob(lower):
            with open(fn, "r", encoding="utf-8") as f:
                contents.append(f.read())
            target = os.path.split(dn)[-1]
            targets.append(target)
    df = pd.DataFrame({
        "content":contents,
        "target":targets
    })
    return df
train_df = read_data("chinese_news_trans")
test_df = read_data("chinese_news_test")

In [9]:
train_df

,content,target
0,《瞭望》週刊文章：一個全球性的環境問題——垃圾\n新華社北京５月１２日...,環境
1,英國確定溫室效應政策\n新華社倫敦５月２５日電為了對付由於溫室效應引起...,環境
2,【 日 期 】19960610\n【 版 號 】10\n【 標 題 】海南人的藍色樂章...,環境
3,經國務院批准我國長江中上游防護林體系建設工程全\n面展開\n (...,環境
4,【 日 期 】19960105\n【 版 號 】7\n【 標 題 】科學家選出２０種最...,環境
...,...,...
2632,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
2633,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
2634,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
2635,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟


In [10]:
test_df

,content,target
0,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
1,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
2,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
3,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
4,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
...,...,...
96,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
97,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
98,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
99,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
